# Spark SQL Lab 1

In [1]:
import findspark
findspark.init()  # uses SPARK_HOME
print("Spark found in : ", findspark.find())

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

# use a unique tmep dir for warehouse dir, so we can run multiple spark sessions in one dir
import tempfile
tmpdir = tempfile.TemporaryDirectory()

config = ( SparkConf()
         .setAppName("TestApp")
         .setMaster("local[*]")
         .set('executor.memory', '2g')
         .set('spark.sql.warehouse.dir', tmpdir.name)
         .set("some_property", "some_value") # another example
         )

spark = SparkSession.builder.config(conf=config).getOrCreate()
print('Spark UI running on port ' + spark.sparkContext.uiWebUrl.split(':')[2])

Spark found in :  /home/sujee/apps/spark
Spark UI running on port 4040


## Read Data Without Schema


In [2]:
df = spark.read.csv('../data/transactions/transactions-sample.csv', header=True, sep="|")
df.show()
df.printSchema()

+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|                  id|           timestamp| mti|     card_number|amount_customer|merchant_type|merchant_id|    merchant_address|              ref_id|amount_merchant|response_code|
+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|59132a14-bdd2-4ab...|2016-08-16 10:22:...|0100|4869275837969573|         105.10|         2788|         63|Sun Shine Liquors...|                null|           null|         null|
|90f04967-f739-439...|2016-08-16 10:22:...|0110|4869275837969573|         105.10|         2788|         63|Sun Shine Liquors...|59132a14-bdd2-4ab...|         101.95|           00|
|dbdf1aa3-bf7e-465...|2018-05-06 04:19:...|0100|4196274028761630|         706.70|         2723|     

## Read Data with Schema

In [3]:
import pyspark
from pyspark.sql.types import ArrayType, IntegerType, LongType, StringType, FloatType, TimestampType, StructType, StructField

my_schema = StructType([
                       StructField("id", StringType(), True),
                       StructField("timestamp", StringType(), True),
                       StructField("mti", StringType(), True),
                       StructField("card_number", StringType(), True),
                       StructField("amount_customer", FloatType(), True),
                       StructField("merchant_type", StringType(), True),
                       StructField("merchant_id", StringType(), True),
                       StructField("merchant_address", StringType(), True),
                       StructField("ref_id", StringType(), True),
                       StructField("amount_merchant", FloatType(), True),
                       StructField("response_code", StringType(), True),
                      ])


df = spark.read.csv('../data/transactions/transactions-sample.csv', header=True, schema=my_schema, sep="|")
df.show()
df.printSchema()

+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|                  id|           timestamp| mti|     card_number|amount_customer|merchant_type|merchant_id|    merchant_address|              ref_id|amount_merchant|response_code|
+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|59132a14-bdd2-4ab...|2016-08-16 10:22:...|0100|4869275837969573|          105.1|         2788|         63|Sun Shine Liquors...|                null|           null|         null|
|90f04967-f739-439...|2016-08-16 10:22:...|0110|4869275837969573|          105.1|         2788|         63|Sun Shine Liquors...|59132a14-bdd2-4ab...|         101.95|           00|
|dbdf1aa3-bf7e-465...|2018-05-06 04:19:...|0100|4196274028761630|          706.7|         2723|     

## Very Simple SQL Query

Let's find the average amount spent on purchases

In [4]:
# first register the dataframe as a temp table

df.createOrReplaceTempView("transactions")

In [5]:
# let's try a simple select query

spark.sql("select * from transactions").show(10)

+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|                  id|           timestamp| mti|     card_number|amount_customer|merchant_type|merchant_id|    merchant_address|              ref_id|amount_merchant|response_code|
+--------------------+--------------------+----+----------------+---------------+-------------+-----------+--------------------+--------------------+---------------+-------------+
|59132a14-bdd2-4ab...|2016-08-16 10:22:...|0100|4869275837969573|          105.1|         2788|         63|Sun Shine Liquors...|                null|           null|         null|
|90f04967-f739-439...|2016-08-16 10:22:...|0110|4869275837969573|          105.1|         2788|         63|Sun Shine Liquors...|59132a14-bdd2-4ab...|         101.95|           00|
|dbdf1aa3-bf7e-465...|2018-05-06 04:19:...|0100|4196274028761630|          706.7|         2723|     

## Another SQL query


### Average charge in credit card

In [6]:
# find average charge

spark.sql ("select AVG(amount_customer) from transactions").show()

+--------------------+
|avg(amount_customer)|
+--------------------+
|  456.52105311343547|
+--------------------+



### Summary by merchant_id

In [7]:
# do a summary on merchant_id

spark.sql ("select merchant_id, COUNT(*) as total from transactions group by merchant_id").show()

+-----------+-----+
|merchant_id|total|
+-----------+-----+
|          7|    2|
|          0|    8|
|         90|    1|
|         58|    2|
|         63|    4|
|         13|    2|
+-----------+-----+



## Querying Large Scale Data

Here we queried sample data - that is very small in size.

In next lab, we will query large datasets.

For large datasets, it is recommended to run the command line clients.

Look at file : [sql-2-query-large-data.py](sql-2-query-large-data.py)